In [12]:
import numpy as np
import keras
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import MaxPooling2D, AveragePooling2D, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
import keras.backend as K
K.set_image_data_format("channels_last")
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import math
import h5py


%matplotlib inline


In [17]:
def load_dataset():
    train_dataset = h5py.File(r"C:\Users\anush\Desktop\happy faces sra\train_happy.h5", "r") #happy house dataset on kaggle
    test_dataset = h5py.File(r"C:\Users\anush\Desktop\happy faces sra\test_happy.h5", "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:])#images
    train_set_y_orig = np.array(train_dataset["train_set_y"][:])#labels
    test_set_x_orig = np.array(test_dataset["test_set_x"][:])
    test_set_y_orig = np.array(test_dataset["test_set_y"][:])
    classes = np.array(test_dataset["list_classes"][:])
    print(train_set_y_orig.shape)
    train_set_y_orig = train_set_y_orig.reshape(1, train_set_y_orig.shape[0])
    test_set_y_orig = test_set_y_orig.reshape(1, test_set_y_orig.shape[0])
    print(train_set_y_orig.shape)
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes
    

In [18]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

# Normalize image vectors
X_train = X_train_orig/255.
X_test = X_test_orig/255.

# Reshape
Y_train = Y_train_orig.T
Y_test = Y_test_orig.T

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

(600,)
(1, 600)
number of training examples = 600
number of test examples = 150
X_train shape: (600, 64, 64, 3)
Y_train shape: (600, 1)
X_test shape: (150, 64, 64, 3)
Y_test shape: (150, 1)


In [19]:
# GRADED FUNCTION: HappyModel

def HappyModel(input_shape):
   
    
    X_input = Input(input_shape)
    
    #Block 1 -- conv2d - MaxPooling
    
    X = X_input
    
    X = Conv2D(30,(3,3), strides = (1,1), padding = 'same', name = 'conv0')(X)
    X = BatchNormalization(axis=3, name = 'bn0')(X)
    X = Activation('relu')(X)
    
    #MaxPool
    
    X = MaxPooling2D((4,4), strides = (2,2), name = 'max_pool')(X)
    
    
    #Block2 -- conv2D -- maxpool2D
    
    X = Conv2D(96,(1,1), name = 'conv1')(X)
    X = BatchNormalization(axis=3, name = 'bn1')(X)
    X = Activation('relu')(X)
    
    #MaxPool
    
    X= MaxPooling2D((3,3), strides = (2,2), name = 'max_pool_1')(X)
    
    
    #Block3 -- conv2D -- MaxPool2D
    
    X = Conv2D(256,(5,5), name ="conv2")(X)
    X = BatchNormalization(axis = 3, name = 'bn2')(X)
    X = Activation("relu")(X)
    
    #MaxPool
    
    X = MaxPooling2D((4,4), strides = (2,2), name = 'max_pool_2')(X)
    
    #Block4 --flatten --dense
    
    X = Flatten()(X)
    X = Dense(1, activation = 'sigmoid', name = 'fc')(X)
    
   
    
     #Block5 
    
    model = Model(inputs = X_input, output = X, name = 'HappyModel')
    
    return model

    #Returns:
    #model -- a Model() instance in Keras
    
    ### START CODE HERE ###
    # Feel free to use the suggested outline in the text above to get started, and run through the whole
    # exercise (including the later portions of this notebook) once. The come back also try out other
    # network architectures as well. 
    

#END CODE HERE

In [ ]:
#prototype model 2

def model1(input_shape):
    
    X_input = Input(input_shape)
    X = ZeroPadding2D((3,3))(X_input)
    
    
    #Block1
    X = Conv2D(96, (7,7), name = 'conv0')(X)
    X = BatchNormalization(axis = 3, name = 'bn0')(X)  #BatchNormalization takes place over channels 
    X = Activation('relu')(X)
    X = MaxPooling2D((4,4), strides = (2,2), name = "max_pool_0")(X)
    
    #Block2 
    X = Conv2D(256, (3,3), strides = (2,2), name = 'conv1')(X)
    X = BatchNormalization(axis = 3, name = 'bn1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3,3), strides = (2,2), name = 'max_pool_1')(X)
    
    #Block3
    X = keras.Dropout(0.5)(X)
    X = Flatten()(X)
    X = Dense(1, activation = 'sigmoid', name = 'fc')(X)
    
    #Block4
    model = Model(inputs = X_input, outputs = X, name = 'HappyModel')
    return model
    


In [20]:
### START CODE HERE ### (1 line)
happyModel = HappyModel(X_train.shape[1:])
### END CODE HERE ###

<ipython-input-19-3d448f2b5305>:51: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., name="HappyModel", outputs=Tensor("fc...)`
  model = Model(inputs = X_input, output = X, name = 'HappyModel')


In [21]:
### START CODE HERE ### (1 line)
happyModel.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])
### END CODE HERE ###

In [22]:
### START CODE HERE ### (1 line)
history = happyModel.fit(X_train, Y_train, epochs = 10, batch_size=10)
### END CODE HERE ###

Epoch 1/10
600/600 [==============================] - 19s 31ms/step - loss: 0.9211 - accuracy: 0.7317
Epoch 2/10
600/600 [==============================] - 16s 26ms/step - loss: 0.2729 - accuracy: 0.8833
Epoch 3/10
600/600 [==============================] - 20s 33ms/step - loss: 0.1402 - accuracy: 0.9417
Epoch 4/10
600/600 [==============================] - 18s 30ms/step - loss: 0.1049 - accuracy: 0.9467
Epoch 5/10
600/600 [==============================] - 19s 31ms/step - loss: 0.0608 - accuracy: 0.9850
Epoch 6/10
600/600 [==============================] - 18s 30ms/step - loss: 0.0247 - accuracy: 0.9967
Epoch 7/10
600/600 [==============================] - 17s 28ms/step - loss: 0.0388 - accuracy: 0.9900
Epoch 8/10
600/600 [==============================] - 16s 27ms/step - loss: 0.0253 - accuracy: 0.9900
Epoch 9/10
600/600 [==============================] - 16s 27ms/step - loss: 0.0358 - accuracy: 0.9900
Epoch 10/10
600/600 [==============================] - 16s 26ms/step - loss: 0.030

In [23]:
### START CODE HERE ### (1 line)
preds = happyModel.evaluate(X_test, Y_test)
### END CODE HERE ###
print(preds)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

150/150 [==============================] - 2s 13ms/step
[0.22284995178381603, 0.9266666769981384]
Loss = 0.22284995178381603
Test Accuracy = 0.9266666769981384


In [ ]:
### START CODE HERE ###
img_path = 'images/my_image.jpg'
### END CODE HERE ###
img = image.load_img(img_path, target_size=(64, 64))
imshow(img)

x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

print(happyModel.predict(x))

In [25]:
happyModel.summary()

Model: "HappyModel"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
conv0 (Conv2D)               (None, 64, 64, 30)        840       
_________________________________________________________________
bn0 (BatchNormalization)     (None, 64, 64, 30)        120       
_________________________________________________________________
activation_1 (Activation)    (None, 64, 64, 30)        0         
_________________________________________________________________
max_pool (MaxPooling2D)      (None, 31, 31, 30)        0         
_________________________________________________________________
conv1 (Conv2D)               (None, 31, 31, 96)        2976      
_________________________________________________________________
bn1 (BatchNormalization)     (None, 31, 31, 96)        3

In [26]:
plot_model(happyModel, to_file='HappyModel.png')
SVG(model_to_dot(happyModel).create(prog='dot', format='svg'))

ImportError: Failed to import `pydot`. Please install `pydot`. For example with `pip install pydot`.